In [2]:
import pandas as pd 
import numpy as np
from utils import *

In [2]:
!ls

LICENSE           README.md         data              exploring_2.ipynb


In [3]:
data = pd.read_csv('data/kidney_disease/kidney_disease.csv')

In [4]:
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [5]:
data.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [39]:
data['pc'].dtype

dtype('O')

In [40]:
type(data['pc'][0])

str

In [16]:
num_variables = []
categ_variables = []

In [22]:
columns = list(data.columns)
n = len(columns)
for i in range(n):
    if data[columns[i]].dtype == 'int' or data[columns[i]].dtype == 'float':
        num_variables.append(columns[i])
    else : 
        categ_variables.append(columns[i])

## Nettoyage données banknote

In [23]:
data2 = pd.read_csv('data/banknote/data_banknote_authentication.txt')

In [25]:
data2.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [26]:
data2.columns

Index(['variance', 'skewness', 'curtosis', 'entropy', 'class'], dtype='object')

,variance,skewness,curtosis,entropy
0,3.621600,8.66610,-2.807300,-0.446990
1,4.545900,8.16740,-2.458600,-1.462100
2,3.866000,-2.63830,1.924200,0.106450
3,3.456600,9.52280,-4.011200,-3.594400
4,0.329240,-4.45520,4.571800,-0.988800
5,4.368400,9.67180,-3.960600,-3.162500
6,3.591200,3.01290,0.728880,0.564210
7,2.092200,-6.81000,8.463600,-0.602160
8,3.203200,5.75880,-0.753450,-0.612510
9,1.535600,9.17720,-2.271800,-0.735350


## Pre-processing

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

Replacing missing values

In [80]:
def detect_type(data):
    data = data.iloc[:,:-1]
    num_variables = []
    categ_variables = []
    columns = list(data.columns)
    n = len(columns)
    for i in range(n):
        if data[columns[i]].dtype == 'int' or data[columns[i]].dtype == 'float':
            num_variables.append(columns[i])
        else : 
            categ_variables.append(columns[i])
    return num_variables, categ_variables

In [87]:
def replace_missing(data, num_variables, categ_variables, num_strategy = 'mean', categ_strategy = 'most_frequent'):
    data = data.iloc[:,:-1]
    ct = ColumnTransformer([("categ_imput", SimpleImputer(missing_values = np.nan, strategy = categ_strategy), categ_variables),
                        ("num_imput", SimpleImputer(missing_values = np.nan, strategy = num_strategy), num_variables)])
    data_transformed = ct.fit_transform(data)
    columns = categ_variables + num_variables
    data_tr_table = pd.DataFrame(data_transformed, columns = columns)
    return data_tr_table

Normalizing and enconding the data

In [88]:
def center_encode(data, num_variables, categ_variables):
    cat_enc = OneHotEncoder()
    center_norm = StandardScaler()
    ct = ColumnTransformer([("categ_encod", cat_enc, categ_variables),
                        ("norm", center_norm, num_variables)])
    data_transformed = ct.fit_transform(data)
    columns = categ_variables + num_variables
    data_tr_table = pd.DataFrame(data_transformed, columns = columns)
    return data_tr_table

In [89]:
num_variables, categ_variables = detect_type(data2)

In [90]:
num_variables

['variance', 'skewness', 'curtosis', 'entropy']

In [97]:
data_tr = replace_missing(data2, num_variables, categ_variables)

In [98]:
data_tr.shape

(1372, 4)

In [99]:
center_encode(data_tr, num_variables, categ_variables)

,variance,skewness,curtosis,entropy
0,1.121806,1.149455,-0.975970,0.354561
1,1.447066,1.064453,-0.895036,-0.128767
2,1.207810,-0.777352,0.122218,0.618073
3,1.063742,1.295478,-1.255397,-1.144029
4,-0.036772,-1.087038,0.736730,0.096587
5,1.384604,1.320874,-1.243653,-0.938386
6,1.111108,0.185881,-0.155217,0.836028
7,0.583612,-1.488408,1.640023,0.280680
8,0.974571,0.653913,-0.499268,0.275752
9,0.387745,1.236571,-0.851680,0.217263
